<a href="https://colab.research.google.com/github/itimes-digital/mineracao-texto/blob/main/analise_de_texto_para_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
import nltk
import pandas as pd
import re

In [186]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [187]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### Preparação dos dados

In [188]:
base = pd.read_csv('sample_data/lista_classificada.csv', sep=';', engine='python')

In [189]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",Nao recebi email para validar o cadastro NÃO C...
1,"Cadastro, Login, Senha",Não consigo concluir o cadastro.
2,"Loja C&R, Estoque",Nunca realizei compras pelo app. Pois a forma ...
3,Entrega,"Muito demora pra entregar.Farmácia é urgência,..."
4,Entrega,"Péssimo no que se refere ao item ENTREGA, pois..."


In [190]:
base['Comentario'] = base['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
base['Comentario'] = base['Comentario'].map(lambda x: x.lower())

In [191]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",nao recebi email para validar o cadastro não c...
1,"Cadastro, Login, Senha",não consigo concluir o cadastro
2,"Loja C&R, Estoque",nunca realizei compras pelo app pois a forma d...
3,Entrega,muito demora pra entregarfarmácia é urgência q...
4,Entrega,péssimo no que se refere ao item entrega pois ...


In [192]:
base['Categoria'].value_counts()

Lentidão, Trava, Crash, Performance    121
Entrega                                 85
Pedido cancelamento                     39
Erro finalização                        38
Erro PIX                                34
Loja C&R, Estoque                       27
Preço, Prazo, Promoção, Stix            21
Erro cartão de crédito                  11
Cesta                                   10
Busca                                    7
Novas Funcionalidades                    6
Cadastro, Login, Senha                   5
Problema não detalhado                   3
Name: Categoria, dtype: int64

In [193]:
base = base.sample(frac=1)

In [194]:
base_treinamento = base.iloc[:350,:]
base_teste = base.iloc[350:,:]

In [195]:
base_treinamento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 228 to 148
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   350 non-null    object
 1   Comentario  350 non-null    object
dtypes: object(2)
memory usage: 8.2+ KB


In [196]:
base_teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 332 to 201
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Categoria   57 non-null     object
 1   Comentario  57 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [197]:
base_treinamento = base_treinamento.reset_index(drop=True)
base_teste = base_teste.reset_index(drop=True)

In [198]:
base_teste.head()

,Categoria,Comentario
0,Pedido cancelamento,cancelam o pedido sem nos ligarnão mostram o p...
1,"Loja C&R, Estoque",muitos erros não consulta estoque antes de ofe...
2,"Preço, Prazo, Promoção, Stix",vou desinstalar o app e usar o site o app dá o...
3,"Preço, Prazo, Promoção, Stix",muito lento na hora de ativar as promoções
4,Entrega,preciso comprar medicamentos controlados para ...


In [199]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [200]:
def removeStopWordsAplicandoStemmer(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords]
        frases.append((sem_stop, base.loc[i,"Categoria"]))
    return frases;

In [201]:
frasesComStemmingTreinamento = removeStopWordsAplicandoStemmer(base_treinamento);
frasesComStemmingTeste = removeStopWordsAplicandoStemmer(base_teste);
#print(frasesComStemming)

In [202]:
def buscaPalavras(frases):
    todasPalavras = [];
    for (palavras, categoria) in frases:
        todasPalavras.extend(palavras);
    return todasPalavras;

In [203]:
palavrasTreinamento = buscaPalavras(frasesComStemmingTreinamento);
palavrasTeste = buscaPalavras(frasesComStemmingTeste);
#print(palavras)

In [204]:
def buscaFrequencia(palavras):
    palavras = nltk.FreqDist(palavras);
    return palavras;

In [205]:
frequenciaTreinamento = buscaFrequencia(palavrasTreinamento);
frequenciaTeste = buscaFrequencia(palavrasTeste);
#print(frequencia.most_common(50))

In [206]:
def buscaPalavrasUnicas(frequencia):
    freq = frequencia.keys();
    return freq;

In [207]:
palavrasUnicasTreinamento = buscaPalavrasUnicas(frequenciaTreinamento);
palavrasUnicasTeste = buscaPalavrasUnicas(frequenciaTeste);
#print(palavrasUnicas)

In [208]:
def extratorPalavras(documento):
    doc = set(documento);
    caracteristicas = {};
    for palavras in palavrasUnicasTreinamento:
        caracteristicas['%s' % palavras] = (palavras in doc);
    return caracteristicas;

In [209]:
caracteristicasFrases = extratorPalavras({'am', 'nov', 'dia'});
#print(caracteristicasFrases)

### Realizando a classificação

In [210]:
baseCompletaTreinamento = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTreinamento);
baseCompletaTeste = nltk.classify.apply_features(extratorPalavras, frasesComStemmingTeste);
#print(baseCompleta);

In [211]:
classificador = nltk.NaiveBayesClassifier.train(baseCompletaTreinamento);

In [212]:
print(classificador.labels())

['Lentidão, Trava, Crash, Performance', 'Entrega', 'Erro cartão de crédito', 'Cesta', 'Loja C&R, Estoque', 'Erro PIX', 'Busca', 'Pedido cancelamento', 'Erro finalização', 'Preço, Prazo, Promoção, Stix', 'Problema não detalhado', 'Cadastro, Login, Senha', 'Novas Funcionalidades']


In [213]:
 #usando a mesma amostra do modelo treinado
print(nltk.classify.accuracy(classificador, baseCompletaTreinamento))

0.6857142857142857


In [214]:
#usando a amostra de teste
print(nltk.classify.accuracy(classificador, baseCompletaTeste)) 

0.5964912280701754


In [215]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                  cancel = True           Pedido : Lentid =     63.2 : 1.0
                    cart = True           Erro c : Entreg =     40.6 : 1.0
                     pix = True           Erro P : Lentid =     36.4 : 1.0
                  entreg = True           Entreg : Lentid =     33.7 : 1.0
                    salv = True           Novas  : Lentid =     33.0 : 1.0
                    cont = True           Cadast : Entreg =     25.7 : 1.0
                   códig = True           Erro P : Lentid =     24.5 : 1.0
                  produt = True           Loja C : Lentid =     23.8 : 1.0
                 encontr = True            Busca : Lentid =     23.6 : 1.0
                  crédit = True           Erro c : Entreg =     23.5 : 1.0
None


### Montagem de uma frase para testar classificação.

In [216]:
testeFrase = 'Não aparece mais o Retirar na Loja!!! Desisti'

In [217]:
def aplicandoStemmerFrase(frase):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for (palavrasTreinamento) in frase.split():
        com_stem_sem_stop = [p for p in palavrasTreinamento.split()];
        frases.append(str(stemmer.stem(com_stem_sem_stop[0])));
    return frases;

In [218]:
fraseComStemming = aplicandoStemmerFrase(testeFrase)
print(fraseComStemming)

['não', 'aparec', 'mais', 'o', 'retir', 'na', 'loja!!!', 'desist']


In [219]:
novaFrase = extratorPalavras(fraseComStemming);
#print(novaFrase)

In [220]:
print(classificador.classify(novaFrase))

Lentidão, Trava, Crash, Performance


In [221]:
distribuicao = classificador.prob_classify(novaFrase)
for classe in distribuicao.samples():
    print("%s: %f" % (classe, distribuicao.prob(classe)))

Lentidão, Trava, Crash, Performance: 0.989156
Entrega: 0.010841
Erro cartão de crédito: 0.000000
Cesta: 0.000000
Loja C&R, Estoque: 0.000000
Erro PIX: 0.000002
Busca: 0.000000
Pedido cancelamento: 0.000000
Erro finalização: 0.000001
Preço, Prazo, Promoção, Stix: 0.000000
Problema não detalhado: 0.000000
Cadastro, Login, Senha: 0.000000
Novas Funcionalidades: 0.000000


### Montagem dos dados novos para classificação

In [222]:
baseSemCategoria = pd.read_csv('sample_data/lista.csv', sep=';', engine='python')

In [223]:
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
baseSemCategoria['Comentario'] = baseSemCategoria['Comentario'].map(lambda x: x.lower())

In [224]:
def removeStopWordsAplicandoStemmerSemCategoria(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords];
        frases.append(sem_stop);
    return frases;

In [225]:
frasesComStemmingSemCategoria = removeStopWordsAplicandoStemmerSemCategoria(baseSemCategoria)
#print(frasesComStemmingSemCategoria)

In [226]:
baseCompletaSemClassificacao = nltk.classify.apply_features(extratorPalavras, frasesComStemmingSemCategoria);

In [227]:
#print(baseCompletaSemClassificacao)